In [1]:
from tracker.centroidtracker import CentroidTracker
from tracker.trackableobject import TrackableObject
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import os

In [2]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0,255,size=(len(CLASSES),3))

In [3]:
prototxt = "mobilenet_ssd/MobileNetSSD_deploy.prototxt"
model = "mobilenet_ssd/MobileNetSSD_deploy.caffemodel"
output = "output/cars2.avi"

writer = None
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(prototxt, model)



print("[INFO] opening video file...")
vs = cv2.VideoCapture("videos/highway.mp4")
W = None
H = None
writer = None



ct = CentroidTracker(maxDisappeared=30, maxDistance=50)
trackers = []
trackableObjects = {}


totalFrames = 0
totalDown = 0
totalUp = 0

fps = FPS().start()


while True:
    
    frame = vs.read()
    frame = frame[1]

    
    if frame is None:
        break

    
    frame = imutils.resize(frame, width=1500)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if W is None or H is None:
        (H, W) = frame.shape[:2]

    if output is not None and writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output, fourcc, 30,
            (W, H), True)

    
    status = "Waiting"
    rects = []

    
    if totalFrames % 5 == 0:
        status = "Detecting"
        trackers = []

        blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5)
        net.setInput(blob)
        detections = net.forward()

        for i in np.arange(0, detections.shape[2]):
            
            confidence = detections[0, 0, i, 2]


            if confidence > 0.4:
                
                idx = int(detections[0, 0, i, 1])

                if CLASSES[idx] != "car":
                    continue

                box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
                (startX, startY, endX, endY) = box.astype("int")

                
                tracker = dlib.correlation_tracker()
                rect = dlib.rectangle(startX, startY, endX, endY)
                tracker.start_track(rgb, rect)

                trackers.append(tracker)


    else:
        for tracker in trackers:
            
            status = "Tracking"

            tracker.update(rgb)
            pos = tracker.get_position()

            startX = int(pos.left())
            startY = int(pos.top())
            endX = int(pos.right())
            endY = int(pos.bottom())

            rects.append((startX, startY, endX, endY))

    
    cv2.line(frame, (0, H // 2), (W, H // 2), (0, 255, 255), 2)

    
    objects = ct.update(rects)

    for (objectID, centroid) in objects.items():
        
        to = trackableObjects.get(objectID, None)

        if to is None:
            to = TrackableObject(objectID, centroid)


        else:
            
            y = [c[1] for c in to.centroids]
            direction = centroid[1] - np.mean(y)
            to.centroids.append(centroid)

            if not to.counted:
                
                if direction < 0 and centroid[1] < H // 2:
                    totalUp += 1
                    to.counted = True

            
                elif direction > 0 and centroid[1] > H // 2:
                    totalDown += 1
                    to.counted = True

        trackableObjects[objectID] = to


        text = "CAR {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)


    info = [
        ("Up", totalUp),
        ("Down", totalDown)
    ]

    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(frame, text, (10, H - ((i * 60) + 40)),
            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)

    if writer is not None:
        writer.write(frame)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

    totalFrames += 1
    fps.update()

fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

if writer is not None:
    writer.release()

vs.release()

cv2.destroyAllWindows()

[INFO] loading model...
[INFO] opening video file...
[INFO] elapsed time: 137.04
[INFO] approx. FPS: 6.64
